In [1]:
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from collections import OrderedDict
from PIL import Image

#GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_dir = 'data/flowers102'

# data augmentation transforms
train_transforms = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224),  # Working with small figures
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2), scale=(0.75, 1.25), shear=20),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3),
    transforms.RandomInvert(p=0.1),
    transforms.RandomSolarize(threshold=192.0, p=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

validation_transforms = transforms.Compose([
    transforms.Resize(180),
    transforms.CenterCrop(180),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#load data
train_dataset = datasets.Flowers102(root=data_dir, split='train', transform=train_transforms, download=True)
validation_dataset = datasets.Flowers102(root=data_dir, split='val', transform=validation_transforms, download=True)
test_dataset = datasets.Flowers102(root=data_dir, split='test', transform=validation_transforms, download=True)


class_idx_mapping = {str(i): i for i in range(102)}
train_dataset.class_to_idx = class_idx_mapping
validation_dataset.class_to_idx = class_idx_mapping
test_dataset.class_to_idx = class_idx_mapping


train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
valid_dataloader = DataLoader(validation_dataset, shuffle=True, batch_size=64, num_workers=4)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=64, num_workers=4)


class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(128 * 22 * 22, 512)),  # Adjusted for 180x180 input size reduced by 3 pooling layers
            ('relu', nn.ReLU(inplace=True)),
            ('dropout', nn.Dropout(p=0.5)),
            ('fc2', nn.Linear(512, 102)),
            ('output', nn.LogSoftmax(dim=1))
        ]))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = CustomCNN()
model.to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def validation(model, testloader, criterion, device):
    test_loss = 0
    accuracy = 0
    model.to(device)
    model.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            test_loss += criterion(output, labels).item()
            ps = torch.exp(output)
            equality = (labels.data == ps.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
    return test_loss, accuracy

def train(model, trainloader, validloader, epochs, print_every, criterion, optimizer, device='cuda'):
    steps = 0
    model.train()
    model.to(device)
    for e in range(epochs):
        running_loss = 0
        for images, labels in trainloader:
            steps += 1
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if steps % print_every == 0:
                model.eval()
                with torch.no_grad():
                    validation_loss, accuracy = validation(model, validloader, criterion, device)
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Validation Loss: {:.3f}.. ".format(validation_loss/len(validloader)),
                      "Validation Accuracy: {:.3f}".format((accuracy/len(validloader))*100))
                model.train()
                running_loss = 0

train(model=model, trainloader=train_dataloader, validloader=valid_dataloader, epochs=500, print_every=20, criterion=criterion, optimizer=optimizer, device=device)

def check_accuracy_on_test(testloader, model):
    correct = 0
    total = 0
    model.to(device)
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

test_accuracy = check_accuracy_on_test(test_dataloader, model)
print('Accuracy of the network on the test images: %d %%' % test_accuracy)

ModuleNotFoundError: No module named 'torch'